# Radar Data Streaming and Pre-Processing

### This notebook demonstrates how to read the data from the radar using python wrapper for Infineon Radar SDK

#### Importing required libraries 

    - ifxdaq --> is the python wrapper for Radar SDK
    - processing --> is python module for basic processing to get the range-Dopplers from the signal
    - fft --> contains the range_fft and doppler_fft methods
    

In [ ]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
import matplotlib.pyplot as plot
from sklearn.preprocessing import MinMaxScaler

#### In the folder 'radar_configs' you find radar configuration parameters. It contains radar parameters :

    - transmit power level
    - start/end frequency
    - sampling rate
    - # of chirps per frame
    - # of samples per chirp
    - # of transmit/receive antennas
    - MIMO mode selection

In [ ]:
"""
@params: sensor's raw data from 3 antenas
@return: Min-Max 
"""
def input_preprocessing(test_data):
    data = np.empty((3,64,128))
    scaler = MinMaxScaler()
    for i in range(3):
        data[i] = scaler.fit_transform(test_data[i])
    return data

In [ ]:
config_file = "radar_configs/RadarIfxBGT60.json"
number_of_frames = 1000

In [ ]:
## Run this to understand the current radar settings better
import json
with open(config_file) as json_file:
    c = json.load(json_file)["device_config"]["fmcw_single_shape"]
    chirp_duration = c["num_samples_per_chirp"]/c['sample_rate_Hz']
    frame_duration = (chirp_duration + c['chirp_repetition_time_s']) * c['num_chirps_per_frame']
    print("With the current configuration, the radar will send out " + str(c['num_chirps_per_frame']) + \
          ' signals with varying frequency ("chirps") between ' + str(c['start_frequency_Hz']/1e9) + " GHz and " + \
          str(c['end_frequency_Hz']/1e9) + " GHz.")
    print('Each chirp will consist of ' + str(c["num_samples_per_chirp"]) + ' ADC measurements of the IF signal ("samples").')
    print('A chirp takes ' + str(chirp_duration*1e6) + ' microseconds and the delay between the chirps is ' + str(c['chirp_repetition_time_s']*1e6) +' microseconds.')
    print('With a total frame duration of ' + str(frame_duration*1e3) + ' milliseconds and a delay of ' + str(c['frame_repetition_time_s']*1e3) + ' milliseconds between the frame we get a frame rate of ' + str(1/(frame_duration + c['frame_repetition_time_s'])) + ' radar frames per second.')

#### the next code block initiliaze the radar with set configuration and loop through the continuous frames; when it  reaches desired number of frames, we process it to get the range-doppler information out of it.

In [ ]:
"""
Loading model.
"""
import tensorflow as tf
model = tf.keras.models.load_model('../my_model_today_data.h5')

In [ ]:

raw_data = []
results = []

"""
We running the monitor procedure in a VM so we have latency in the predictions
"""
with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        
        inference_data = input_preprocessing(raw_data[len(raw_data)-1])
        inference_data = inference_data.reshape(-1,64,128,3)
        prediction = model.predict(inference_data,verbose=0)
        results.append(np.argmax(prediction))
        print('\r People prediction: ' + str(np.argmax(prediction)), end='\r')

        if(i_frame == number_of_frames-1):
            data = np.asarray(raw_data)
            range_doppler_map = processing.processing_rangeDopplerData(data)
            # print(loaded_model.predict([range_doppler_map]))
            #del data
            break      

In [ ]:
"""
Data acquition
"""
raw_data    = []

with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data

        if(i_frame == number_of_frames-1):
            data = np.asarray(raw_data)
            range_doppler_map = processing.processing_rangeDopplerData(data)
            # print(loaded_model.predict([range_doppler_map]))
            #del data
            break     

#### Note: There is a function to get range_data called "processing_rangeData" which will be usefull to detect the movements.

In [ ]:
# The shape of the radar data depends on the radar settings.
# The different dimensions of the shape are (frame_index, receiver_index, chirp_index, sample_index).

print("(frame_index, receiver_index, chirp_index, sample_index)")
print(data.shape)

In [ ]:
"""
Saving data
"""
np.save('dataset1',raw_data)